# URSA 3D Tucker Decomposition of Dataset
### Goal: Discover important minerals within dataset
Left all 0 values in dataset during cleaning.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorly.decomposition import tucker, non_negative_tucker
import tensorly as tl

data = pd.read_csv("../data_files/cleaned_mesh_react.txt", delimiter=',', index_col=['TIME', 'INDEX'])
print('Shape: ', data.shape, '\n\nColumns: ', data.columns)

Shape:  (75642, 57) 

Columns:  Index(['ELEM_x', 'P (Pa)', 'T (deg C)', 'SG', 'SS', 'XNaCl', 'YH2OG', 'XCO2aq',
       'PCAP (Pa)', 'k-red', 'DG (kg/m^3)', 'DL (kg/m^3)', 'PER MOD', 'X2',
       'X3', 'DX1', 'DX2', 'DX3', 'DX4', 'K(gas)', 'K(liq)', 'VIS(gas)',
       'VIS(liq)', 'ELEM_y', 'Sg', 'Sl', 'T', 'pH', 'alo2-', 'ca+2', 'cl-',
       'fe+2', 'h+', 'h2o', 'hco3-', 'k+', 'mg+2', 'na+', 'o2(aq)', 'sio2(aq)',
       'so4-2', 'SMco2', 'Porosity', 'Permeabi.', 'calcite', 'quartz',
       'chlorite', 'illite', 'albite', 'siderite-2', 'ankerite', 'kaolinite',
       'magnesite', 'dolomite', 'smectite-na', 'co2(g)', 'fugcoe_co2(g)'],
      dtype='object')


>Set data into 3D NumPy array for tensor

In [4]:
time_1 = data.xs(31536000)
time_2 = data.xs(102000000)
time_1 = np.array(time_1, dtype=tl.float32)
time_2 = np.array(time_2, dtype=tl.float32)
print(time_1.shape)
print(time_2.shape)

tensor = np.dstack((time_1, time_2))
print(tensor.shape)

# ensure non-negative values
print(np.any(tensor < 0))
tensor[tensor<0] = 0.000001
print(np.any(tensor < 0))

tensor = tl.tensor(tensor)

tl.context(tensor)


(37821, 57)
(37821, 57)
(37821, 57, 2)
True
False


{'dtype': dtype('float32')}

In [5]:
core, factors = non_negative_tucker(tensor, ranks=[37821, 57, 2], rank=3)
print(core.shape)

(37821, 57, 2)


In [6]:
print(core)

[[[4.25572275e+09 4.05394836e+02]
  [1.93534805e+02 1.83149062e+04]
  [2.41579437e+02 4.80944570e+04]
  ...
  [9.37418765e-09 6.76884451e-11]
  [1.68332033e-08 4.84418148e-12]
  [1.83132076e+00 3.25226517e+01]]

 [[4.82979059e+00 2.01867940e+07]
  [1.11870609e+05 5.35996758e+04]
  [3.57992891e+04 4.13426211e+04]
  ...
  [2.23494681e-10 7.91255741e-11]
  [1.73261822e-10 1.80152560e-10]
  [1.96033134e+01 5.55933619e+00]]

 [[2.18586792e+02 4.00003516e+04]
  [7.30764050e+06 1.17804912e+04]
  [1.29822156e+03 3.26029663e+03]
  ...
  [1.83118889e-08 6.68686851e-12]
  [1.77352959e-08 3.30322922e-11]
  [1.70269632e+00 3.82770419e-01]]

 ...

 [[7.77608824e+00 1.42664015e+00]
  [1.16227195e-02 4.65003680e-03]
  [1.60567855e-04 4.32619345e-05]
  ...
  [1.21354031e-13 1.55578421e-14]
  [2.60432816e-13 4.91126847e-14]
  [3.91601759e-07 2.98021973e-06]]

 [[2.83188844e+00 2.11682510e+00]
  [4.37294366e-05 6.83092326e-03]
  [9.23376356e-05 3.25927758e-05]
  ...
  [1.06887386e-13 1.83316243e-14]
  [2

In [7]:
full_tensor = tl.tucker_to_tensor(core, factors)
print(full_tensor.shape)
print(full_tensor)

(37821, 57, 2)
[[[3.04281074e+04 3.05267461e+04]
  [1.56669730e+07 1.57165570e+07]
  [7.40691071e+01 7.43323059e+01]
  ...
  [5.42140299e-07 5.44537841e-07]
  [8.43224106e+01 8.47006836e+01]
  [6.27730787e-01 6.30887449e-01]]

 [[3.05169238e+04 3.06158418e+04]
  [1.57127490e+07 1.57624730e+07]
  [7.87481613e+01 7.90852661e+01]
  ...
  [5.92828485e-07 5.95940662e-07]
  [8.48610687e+01 8.52483902e+01]
  [6.84558034e-01 6.88541710e-01]]

 [[3.06044590e+04 3.07036660e+04]
  [1.57576950e+07 1.58075640e+07]
  [7.61634369e+01 7.64493484e+01]
  ...
  [5.54727194e-07 5.57267128e-07]
  [8.48884811e+01 8.52683029e+01]
  [6.47951543e-01 6.51337385e-01]]

 ...

 [[5.18355781e+04 5.20018633e+04]
  [1.48890870e+07 1.49362170e+07]
  [7.30682144e+01 7.33371353e+01]
  ...
  [9.44370697e-07 9.47798696e-07]
  [1.21400146e+02 1.21911209e+02]
  [7.73617148e-01 7.77290046e-01]]

 [[5.18651836e+04 5.20315586e+04]
  [1.48552400e+07 1.49022640e+07]
  [7.30873566e+01 7.33565826e+01]
  ...
  [9.44285432e-07 9.477

>Trying with different ranks

In [8]:
core, factors = non_negative_tucker(tensor, ranks=[37821, 53, 2], rank=3)
print(core.shape)
print(core)

C:\ProgramData\Anaconda3\lib\site-packages\tensorly\backend\numpy_backend.py:241: RuntimeWarning: invalid value encountered in sqrt
  S = np.sqrt(S)


(37821, 53, 2)
[[[4.2557207e+09 2.7082355e+02]
  [4.1232584e+02 1.8121791e+04]
  [8.7167969e+01 1.6744410e+04]
  ...
  [1.9568405e+02 1.5675890e+00]
  [3.3743080e+01 4.1612806e+00]
  [1.6310066e+01 4.9030328e+00]]

 [[1.1018087e+01 2.0186900e+07]
  [1.1163821e+05 5.3374457e+04]
  [1.5777263e+04 1.6705109e+04]
  ...
  [3.9038835e+00 1.0333112e+00]
  [3.5458630e-01 4.8307643e+00]
  [1.5914819e+02 6.6407539e+01]]

 [[3.9735638e+02 3.9795664e+04]
  [7.3077505e+06 1.1792812e+04]
  [8.4089188e+04 1.0805927e+03]
  ...
  [5.7381991e-02 2.3057835e-02]
  [7.7138394e-02 9.1442862e+00]
  [3.5837133e+00 1.3508250e+01]]

 ...

 [[7.7817407e+00 2.1089242e+00]
  [1.1644685e-02 6.9235140e-03]
  [6.8590150e-04 1.6905223e-04]
  ...
  [3.0778692e-04 5.8717933e-06]
  [2.3428353e-03 7.7869436e-07]
  [7.3161682e-05 3.8963211e-05]]

 [[7.0715147e-01 1.1284751e-03]
  [1.1440684e-02 1.7907105e-05]
  [3.4204469e-04 2.5646586e-05]
  ...
  [1.8670678e-05 4.5239899e-06]
  [1.1917109e-03 7.7284039e-06]
  [2.7392778e

In [16]:
full_tensor = tl.tucker_to_tensor(core, factors)
print(full_tensor.shape)
print(full_tensor)

(2, 37821, 57)
[[[3.03469707e+04 1.56669760e+07 5.93810997e+01 ... 2.63893156e-07
   7.25069504e+01 3.16838831e-01]
  [3.04358711e+04 1.57127480e+07 5.95072441e+01 ... 2.64741431e-07
   7.27416534e+01 3.18061948e-01]
  [3.05221875e+04 1.57576880e+07 5.97810478e+01 ... 2.65686708e-07
   7.29629440e+01 3.19061458e-01]
  ...
  [5.18713320e+04 1.48890990e+07 7.87884216e+01 ... 7.41015526e-07
   1.17480934e+02 8.63120973e-01]
  [5.19012266e+04 1.48552560e+07 7.87937317e+01 ... 7.42691157e-07
   1.17403809e+02 8.64792466e-01]
  [5.19126602e+04 1.48430570e+07 7.87310257e+01 ... 7.43088208e-07
   1.17371758e+02 8.65283370e-01]]

 [[3.04459883e+04 1.57165460e+07 5.96881409e+01 ... 2.65940145e-07
   7.28526917e+01 3.19567055e-01]
  [3.05351738e+04 1.57624620e+07 5.98133202e+01 ... 2.66790948e-07
   7.30886230e+01 3.20798248e-01]
  [3.06217793e+04 1.58075470e+07 6.00921745e+01 ... 2.67749499e-07
   7.33111191e+01 3.21808994e-01]
  ...
  [5.20385039e+04 1.49362290e+07 7.91691208e+01 ... 7.44783904